In [1]:
!pip show tensorflow

Name: tensorflow
Version: 2.17.1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine_rl, tf_keras


In [2]:
!pip install keras tensorflow

In [3]:
!pip install --upgrade tensorflow keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 107.8 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
  Attempting uninstall: keras
    Found existing installation: keras 3.5.0
    Uninstalling keras-3.5.0:
      Successfully uninstalled keras-3.5.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.1
    Uninstalling tensorflow-2.17.1:
      Successfully uninstalled tensorflow-2.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompa

In [4]:
from tensorflow.keras.utils import to_categorical

In [5]:
# Importações Necessárias
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.utils import to_categorical  # Importação adicionada
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [6]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Carregar o Dataset
file_path = "/content/drive/MyDrive/texto9.txt"
with open(file_path, 'r', encoding='utf-8') as file:
    corpus = file.read()

# 2. Pré-processamento
corpus = corpus.lower()
chars = sorted(list(set(corpus)))  # Lista de caracteres únicos
char_to_index = {c: i for i, c in enumerate(chars)}
index_to_char = {i: c for i, c in enumerate(chars)}

# Converter texto para índices
text_as_int = np.array([char_to_index[c] for c in corpus if c in chars])

# Criar sequências para treinamento
seq_length = 100
sequences = []
for i in range(len(text_as_int) - seq_length):
    seq = text_as_int[i:i + seq_length]
    sequences.append(seq)

sequences = np.array(sequences)
X = sequences[:, :-1]  # Entradas
y = sequences[:, -1]  # Saídas (próximo caractere)
y = to_categorical(y, num_classes=len(chars))  # One-hot encoding das saídas

# 3. Construção do Modelo LSTM
model = Sequential([
    Embedding(input_dim=len(chars), output_dim=50, input_length=seq_length - 1),
    LSTM(150, return_sequences=True),
    LSTM(150),
    Dense(len(chars), activation='softmax')
])

# 4. Compilação do Modelo
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.summary()

# 5. Treinamento
epochs = 150
batch_size = 64
model.fit(X, y, epochs=epochs, batch_size=batch_size)

# 6. Função para Gerar Texto
def generate_text(model, start_string, gen_length=500):
    input_eval = [char_to_index[s] for s in start_string.lower() if s in chars]
    input_eval = pad_sequences([input_eval], maxlen=seq_length - 1, truncating='pre')

    generated_text = start_string
    for _ in range(gen_length):
        predictions = model.predict(input_eval)
        next_index = np.argmax(predictions)
        next_char = index_to_char[next_index]

        generated_text += next_char
        input_eval = np.append(input_eval[0][1:], next_index).reshape(1, -1)

    return generated_text

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/150
431/431 ━━━━━━━━━━━━━━━━━━━━ 11s 15ms/step - loss: 2.9466
Epoch 2/150
431/431 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - loss: 2.2798
Epoch 3/150
431/431 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - loss: 2.1183
Epoch 4/150
431/431 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - loss: 2.0165
Epoch 5/150
431/431 ━━━━━━━━━━━━━━━━━━━━ 11s 15ms/step - loss: 1.9071
Epoch 6/150
431/431 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - loss: 1.8246
Epoch 7/150
431/431 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - loss: 1.7241
Epoch 8/150
431/431 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - loss: 1.6666
Epoch 9/150
431/431 ━━━━━━━━━━━━━━━━━━━━ 11s 15ms/step - loss: 1.5970
Epoch 10/150
431/431 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - loss: 1.5211
Epoch 11/150
431/431 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - loss: 1.4585
Epoch 12/150
431/431 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - loss: 1.3951
Epoch 13/150
431/431 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - loss: 1.3341
Epoch 14/150
431/431 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - loss: 1.2634
Epoch 15/150
431/431

In [7]:
!pip install gtts

In [9]:
# Importar biblioteca para conversão de texto em fala
from gtts import gTTS
import IPython.display as ipd

# 7. Gerar Texto
start_string = "Flamengo"
generated_text = generate_text(model, start_string=start_string, gen_length=500)
print("Texto Gerado:\n", generated_text)

# 8. Converter texto gerado em áudio
tts = gTTS(generated_text, lang='pt')  # Define o texto gerado e o idioma (Português)
audio_path = "/content/drive/MyDrive/generated_audio.mp3"
tts.save(audio_path)  # Salva o áudio no Google Drive
print(f"Áudio salvo em: {audio_path}")

# Reproduzir o áudio no Colab
ipd.Audio(audio_path)  # Gera o player de áudio no notebook

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━